In [6]:
import pandas as pd
import scipy.stats as stats
from constants import questions 

In [7]:
allData = pd.read_csv("LHIPA.csv")

In [8]:
def addQuestionInfo(allData,questions):
    
    #change the type of questionID to integer
    allData['task'] = allData['task'].astype('int')
    
    #extend the columns of questionnaireData with those in DataFrame(questions) based the common question ID
    allData = allData.merge(pd.DataFrame(questions), left_on=['task'], right_on=['id'])
    
    
    return allData

In [9]:
#Add question info
allDataWithQs = addQuestionInfo(allData,questions)
#print(allDataWithQs.shape)
allDataWithQs

,Unnamed: 0,LHIPA,participant,task,id,question,type,options,model-group,ExpectedAnswer,Type1,Type2,Type3
0,0,1.436563,SP10-no,7,7,"The activity ""Enter container information fro...",multiple-choice,True;False;I don't know,1,False,Local,Control-flow,Ordering
1,0,0.800090,SP2-no,7,7,"The activity ""Enter container information fro...",multiple-choice,True;False;I don't know,1,False,Local,Control-flow,Ordering
2,0,1.368287,SP14-no,7,7,"The activity ""Enter container information fro...",multiple-choice,True;False;I don't know,1,False,Local,Control-flow,Ordering
3,0,1.465562,SP6-no,7,7,"The activity ""Enter container information fro...",multiple-choice,True;False;I don't know,1,False,Local,Control-flow,Ordering
4,0,1.425710,SP15-no,7,7,"The activity ""Enter container information fro...",multiple-choice,True;False;I don't know,1,False,Local,Control-flow,Ordering
...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,0,0.830783,KP5-no,46,46,"If the ""Container"" object is in state ""[tempo...",multiple-choice,True;False;I don't know,1,True,Global,Data-flow,Data-based Routing
612,0,0.786943,KP18-no,46,46,"If the ""Container"" object is in state ""[tempo...",multiple-choice,True;False;I don't know,1,True,Global,Data-flow,Data-based Routing
613,0,0.778445,KP4-no,46,46,"If the ""Container"" object is in state ""[tempo...",multiple-choice,True;False;I don't know,1,True,Global,Data-flow,Data-based Routing
614,0,0.722764,KP19-no,46,46,"If the ""Container"" object is in state ""[tempo...",multiple-choice,True;False;I don't know,1,True,Global,Data-flow,Data-based Routing


In [10]:
#Remove SP20 + all questions for a participant (local + global!) if either local or global is nan:
#KP16 --> Ordering
#SP20 --> all
#SP11 --> Exclusivness
#SP2 --> Repetition + Concurrency

measureAllData_new = allDataWithQs.drop(allDataWithQs[allDataWithQs['participant'] == 'SP20-no'].index)

measureAllData_new = measureAllData_new.drop(allDataWithQs[(allDataWithQs['participant'] == 'KP16-no') & (allDataWithQs['Type3'] == 'Ordering')].index)
measureAllData_new = measureAllData_new.drop(allDataWithQs[(allDataWithQs['participant'] == 'SP11-no') & (allDataWithQs['Type3'] == 'Exclusiveness')].index)
measureAllData_new = measureAllData_new.drop(allDataWithQs[(allDataWithQs['participant'] == 'SP2-no') & (allDataWithQs['Type3'] == 'Repetition')].index)
measureAllData_new = measureAllData_new.drop(allDataWithQs[(allDataWithQs['participant'] == 'SP2-no') & (allDataWithQs['Type3'] == 'Concurrency')].index)

In [11]:
#Select control-flow question type
measureAllData_new = measureAllData_new.loc[(measureAllData_new['Type2'] == 'Control-flow')] 

In [17]:
#Check whether the LHIPA is always larger for global than for local tasks for each participant on average
LHIPAPartCompare = measureAllData_new.groupby(['participant','Type1','Type2'],as_index=False).agg({'LHIPA':'mean'})
LHIPAPartCompare_G = LHIPAPartCompare.loc[LHIPAPartCompare['Type1'] == 'Global']
LHIPAPartCompare_L = LHIPAPartCompare.loc[LHIPAPartCompare['Type1'] == 'Local']
LHIPAPartCompare_G_L = pd.merge(LHIPAPartCompare_G, LHIPAPartCompare_L, on='participant')

LHIPAPartCompare_G_L['Glo > Loc'] = LHIPAPartCompare_G_L.apply(lambda x: True if x['LHIPA_x'] < x['LHIPA_y'] else (x['LHIPA_x'],x['LHIPA_y']), axis=1)
LHIPAPartCompare_G_L

,participant,Type1_x,Type2_x,LHIPA_x,Type1_y,Type2_y,LHIPA_y,Glo > Loc
0,KP1-no,Global,Control-flow,0.998822,Local,Control-flow,0.977516,"(0.9988215244161478, 0.977515936997543)"
1,KP10-no,Global,Control-flow,0.715306,Local,Control-flow,1.522794,True
2,KP11-no,Global,Control-flow,1.116338,Local,Control-flow,1.083964,"(1.1163382563947728, 1.083963735696072)"
3,KP12-no,Global,Control-flow,1.133361,Local,Control-flow,1.565494,True
4,KP13-no,Global,Control-flow,0.560528,Local,Control-flow,1.064391,True
5,KP14-no,Global,Control-flow,0.598355,Local,Control-flow,1.091184,True
6,KP15-no,Global,Control-flow,0.307933,Local,Control-flow,0.654738,True
7,KP16-no,Global,Control-flow,0.627983,Local,Control-flow,1.287151,True
8,KP17-no,Global,Control-flow,0.654586,Local,Control-flow,1.121428,True
9,KP18-no,Global,Control-flow,0.734753,Local,Control-flow,0.938462,True


In [12]:
#LHIPA

In [13]:
#calculate mean per participant
LHIPAPart = measureAllData_new.groupby(['participant','Type1','Type2'],as_index=False).agg({'LHIPA':'mean'})
#calculate descriptives
LHIPAPart.groupby(['Type1']).agg({'LHIPA':'mean'})

,LHIPA
Type1,
Global,0.815335
Local,1.185003


In [15]:
LHIPAPart.groupby(['Type1']).agg({'LHIPA':'std'})

,LHIPA
Type1,
Global,0.215667
Local,0.297898


In [14]:
#Inferentials
#Control-flow
lhipa_participant = measureAllData_new.groupby(['participant', 'Type1', 'Type2'],as_index=False).agg({'LHIPA':'mean'})
lhipa_participant = lhipa_participant.reset_index()
#print(lhipa_participant)

lhipa_Global = lhipa_participant.loc[(lhipa_participant["Type1"]=='Global')][['participant','LHIPA']]
lhipa_Local = lhipa_participant.loc[(lhipa_participant["Type1"]=='Local')][['participant','LHIPA']]
print(len(lhipa_Global), len(lhipa_Local))

measure_GL_merge = lhipa_Global.merge(lhipa_Local, on=['participant'], suffixes=('_global', '_local'), how='inner')

print(stats.wilcoxon(measure_GL_merge['LHIPA_global'], measure_GL_merge['LHIPA_local'],alternative='less'))
        

43 43
WilcoxonResult(statistic=29.0, pvalue=1.9770141079789028e-10)
